In [45]:
import os
import pandas as pd
import numpy as np

In [299]:
from typing import Any, Callable, Iterable, Optional, Sequence, Union

In [300]:
from nltk.metrics import masi_distance

In [307]:
from krippendorff.krippendorff import _coincidences

In [ ]:
РАСХОЖДЕНИЕ ИЗ -ЗА РАЗНЫХ МЕТРИК ДИСТАНЦИИ И СООТВЕТСКНЕО ПЛОЗО КОГДА МНОГО ИЗМЕРЕНИЙ СТАНОВИТСЯ

In [310]:
def alpha(reliability_data: Optional[Iterable[Any]] = None, value_counts: Optional[np.ndarray] = None,
          value_domain: Optional[Sequence[Any]] = None,
           dtype: Any = np.float64) -> float:

    if (reliability_data is None) == (value_counts is None):
        raise ValueError("Either reliability_data or value_counts must be provided, but not both.")

    # Don't know if it's a list or numpy array. If it's the latter, the truth value is ambiguous. So, ask for None.
    if value_counts is None:
        reliability_data = np.asarray(reliability_data)

        if value_domain is None:
            value_domain = np.unique(reliability_data[~np.isnan(reliability_data)])
        else:
            value_domain = np.asarray(value_domain)
            assert np.isin(reliability_data, np.append(value_domain, np.nan)).all(), \
                "The reliability data contains out-of-domain values."

        value_counts = _reliability_data_to_value_counts(reliability_data, value_domain)
    else:  # elif reliability_data is None
        value_counts = np.asarray(value_counts)

        if value_domain is None:
            value_domain = np.arange(value_counts.shape[1])
        else:
            value_domain = np.asarray(value_domain)
            assert value_counts.shape[1] == len(value_domain), \
                "The value domain should be equal to the number of columns of value_counts."

    assert len(value_domain) > 1, "There has to be more than one value in the domain."

    distance_metric = masi_distance

    o = _coincidences(value_counts, dtype=dtype)
    n_v = o.sum(axis=0)
    e = _random_coincidences(n_v, dtype=dtype)
    d = _distances(value_domain, distance_metric, n_v, dtype=dtype)
    return 1 - (o * d).sum() / (e * d).sum()

value_counts = np.array([[1, 0, 0, 0],[0, 0, 0, 0]])
alpha(value_counts=value_counts)

TypeError: _coincidences() missing 1 required positional argument: 'value_domain'

In [4]:
# https://stats.stackexchange.com/questions/407453/krippendorffs-alpha-in-r-for-multi-label-annotation
import nltk
from nltk.metrics import agreement
from nltk.metrics.agreement import AnnotationTask


task_data = [('coder1','Item0',frozenset(['l1','l2'])),
('coder2','Item0',frozenset(['l1'])),
('coder1','Item1',frozenset(['l1','l2'])),
('coder2','Item1',frozenset(['l1','l2'])),
('coder1','Item2',frozenset(['l1'])),
('coder2','Item2',frozenset(['l1']))]

task = AnnotationTask(distance = masi_distance)

task.load_array(task_data)

task.alpha()

0.4444444444444444

In [311]:
def calc_agreement_within_group(df_group):
    worker2id = {}
    for pair in df_group['ASSIGNMENT:worker_id'].tolist():
        if pair not in worker2id:
            worker2id[pair] = len(worker2id)
    df_group['worker_idx'] = df_group['ASSIGNMENT:worker_id'].map(worker2id)
    
    text2ids = {}
    for t in set(df_group['INPUT:text'].tolist()):
        if t not in text2ids:
            text2ids[t] = len(text2ids)
    df_group['text_idx'] = df_group['INPUT:text'].map(text2ids)
    ids2text = {idx:text for text,idx in text2ids.items()}
    
    reply_columns = [c for c in df_group.columns if 'OUTPUT' in c]
    
    collected_data = []
    for i,el in df_group.iterrows():
        coder_idx = el['worker_idx']
        
        item_idx = el['text_idx']
        labels_list = []
        for repl in reply_columns:
#             print(repl[7:], el[repl])
            if el[repl] == True:
                labels_list.append(repl[7:])
        
        if len(labels_list) == 0: labels_list = ['none']
#         print(labels_list)
        collected_triplet = (coder_idx,item_idx,frozenset(labels_list))
        collected_data.append(collected_triplet)
        
#         collected_texts.append(el['INPUT:text'])
        
#         break
    task = AnnotationTask(distance = masi_distance)
    
#     print("collected_data", collected_data[:10])

    task.load_array(collected_data)

    return task.alpha()


def get_agreement_batch(batch_name, drop_traintest_examples = False, print_output = False,):
    if batch_name == 'first': 
        raw_files = ['g1r.tsv','g2r.tsv','g3r.tsv']
    elif batch_name == 'fourth':
        raw_files = ['g1_4r.tsv','g2_4r.tsv','g3_4r.tsv']
    elif batch_name == 'old':
        raw_files = ['maybe_old/g1r.tsv','maybe_old/g2r.tsv']
    
    index2textset_notrest = {}
    index2textset_withtrtest = {}
    for file in raw_files:
       
        batch_path = "./directly_from_toloka/"+file
        df_curr= pd.read_csv(batch_path, sep = '\t')
        
        index2textset_withtrtest[file] = set(df_curr['INPUT:text'])
        
        if drop_traintest_examples == True:
            golden_column = [c for c in df_curr.columns if 'GOLDEN' in c]
            df_curr = df_curr[df_curr[golden_column[1]].isna()]
        
        index2textset_notrest[file] = set(df_curr['INPUT:text'])
    
    intersect_text_no_tr_test = index2textset_notrest[file]
    engaged_text_withtrtest = index2textset_withtrtest[file]
    for f in index2textset_notrest:
        intersect_text_no_tr_test = intersect_text_no_tr_test.intersection(index2textset_notrest[f])
        engaged_text_withtrtest = engaged_text_withtrtest.union(index2textset_withtrtest[f])
        
    if print_output == True: print("intersecting texts are {} \n".format(len(intersect_text)))
    
    collected_agreements = []
    for i, file in enumerate(raw_files):
        batch_path = "./directly_from_toloka/"+file
        df_curr_common= pd.read_csv(batch_path, sep = '\t')
        
        if drop_traintest_examples == True:
            golden_column = [c for c in df_curr_common.columns if 'GOLDEN' in c]
            df_curr_common = df_curr_common[df_curr_common[golden_column[1]].isna()]
        
        columns_to_merge = [c for c in df_curr_common.columns if c == 'INPUT:text' or 'OUTPUT' in c  or c =='ASSIGNMENT:worker_id']
        df_curr_common = df_curr_common[columns_to_merge]
        
        current_aggr = calc_agreement_within_group(df_curr_common)
#         print("current_aggr", current_aggr)
        collected_agreements.append(current_aggr)
        
#         break
    if print_output == True: 
        print("\n")
        print("collected_agreements", collected_agreements)
        print("average agreements", np.mean(collected_agreements))
    return intersect_text_no_tr_test, engaged_text_withtrtest, collected_agreements#intersect_text_no_tr_test - с тренй-тест если выбрать опцию не дропать тртест

    
int_text, eng_text, afrr = get_agreement_batch_no_intersect('first')

In [312]:
afrr

[0.10661798648012055, 0.6364950196276913, 0.3874856997131322]

In [329]:
def get_whole_agreement_and_intersections(drop_traintest_examples):
    intersected_texts_from_batches = set()
    all_unique_texts_from_batches =  set()
    toloka_agreements_list = []
    for batch in ['first','fourth','old']:#
        intersect_texts, all_unique_texts_set, aggr_list = get_agreement_batch(batch, drop_traintest_examples = drop_traintest_examples)
        intersected_texts_from_batches = intersected_texts_from_batches.union(intersect_texts)
        all_unique_texts_from_batches = all_unique_texts_from_batches.union(all_unique_texts_set)
        print("{} batch agreement list - {}".format(batch, aggr_list))
        toloka_agreements_list.extend(aggr_list)
    return intersected_texts_from_batches,all_unique_texts_from_batches, np.mean(toloka_agreements_list)
common_texts_whole_toloka_notrtes, all_engaged_texts, whole_toloka_agreement = get_whole_agreement_and_intersections(True)

first batch agreement list - [0.10661798648012055, 0.6364950196276913, 0.3874856997131322]
fourth batch agreement list - [0.4455289036288479, 0.5516957933012958, 0.6099929668782522]
old batch agreement list - [0.41087633393756817, 0.43702795130017413]


In [330]:
# [0.10661798648012055, 0.6364950196276913, 0.3874856997131322] - без трейн-тест семплов

In [331]:
len(common_texts_whole_toloka_notrtes), len(all_engaged_texts), whole_toloka_agreement

(6732, 10130, 0.44821508185838527)

In [332]:
common_texts_whole_toloka_withtrtest,  all_engaged_texts, whole_toloka_agreement = get_whole_agreement_and_intersections(drop_traintest_examples = False)

first batch agreement list - [0.7004942315456364, 0.6767382024830249, 0.5799682486670625]
fourth batch agreement list - [0.5321843310977421, 0.5966316162108932, 0.6597914632811841]
old batch agreement list - [0.4142966632292012, 0.4489685586640344]


In [333]:
len(common_texts_whole_toloka_withtrtest), len(all_engaged_texts), whole_toloka_agreement

(6798, 10130, 0.5761341643973473)

In [334]:
len(common_texts_whole_toloka_withtrtest), len(all_engaged_texts), whole_toloka_agreement

(6798, 10130, 0.5761341643973473)

# Проверяем пересечение текстов фактических батчей с сырой толоки с текстами заявленными в спарсенном датасете

In [335]:
import re
def get_len(text):
    if len(text.split()) > 5 and len (text)< 250:
        return True
    return None
def is_mostly_russian(text):
    text = str(text)
    russian_letters_count = len(re.findall("[а-яА-Я]",text))
    eng_letters_count = len(re.findall("[a-zA-Z]",text))
    if russian_letters_count > eng_letters_count:
        return True
    return None
def depersonalize(text):
    text = str(text)
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text= re.sub(url_regex, "url", text)
    text = re.sub("id[\d]*",'',text)
    text = re.sub("> ?",'',text)
    text = re.sub("@[\w]*",'',text)
#     text = re.sub("\+[\d]*", "", text)
    text = re.sub("[\d]+", "NUMBER", text)
    text = re.sub(">>", "", text)
#     text = re.sub("[\d]{3,100}", "", text)
    text = re.sub("[\t|\n|\r]", "", text)
    text = re.sub("[(OP)|url|nickname|phone_number]", "", text)
    return text

In [336]:
df_sensitive = pd.read_csv("sensitive_topics_vs_source.csv")

In [337]:
df_sensitive_toloka = df_sensitive[df_sensitive['source'] == 'toloka']

In [338]:
len(df_sensitive_toloka)

9654

In [339]:
df_sensitive_toloka_texts_set = set(df_sensitive_toloka['text'])
len(df_sensitive_toloka_texts_set)

9654

### если дропать трейн тест

In [340]:
common_texts_whole_toloka_notrtes, all_engaged_texts, whole_toloka_agreement = get_whole_agreement_and_intersections(True)

first batch agreement list - [0.10661798648012055, 0.6364950196276913, 0.3874856997131322]
fourth batch agreement list - [0.4455289036288479, 0.5516957933012958, 0.6099929668782522]
old batch agreement list - [0.41087633393756817, 0.43702795130017413]


In [341]:
common_texts_whole_toloka_prep = set([depersonalize(t) for t in common_texts_whole_toloka_notrtes])

In [342]:
len(df_sensitive_toloka_texts_set & common_texts_whole_toloka_prep)

6061

In [343]:
len(df_sensitive_toloka_texts_set & common_texts_whole_toloka_prep)/len(df_sensitive_toloka_texts_set)

0.6278226641806505

### если НЕ дропать трейн тест

In [344]:
common_texts_whole_toloka_withtrtest,  all_engaged_texts, whole_toloka_agreement = get_whole_agreement_and_intersections(drop_traintest_examples = False)

first batch agreement list - [0.7004942315456364, 0.6767382024830249, 0.5799682486670625]
fourth batch agreement list - [0.5321843310977421, 0.5966316162108932, 0.6597914632811841]
old batch agreement list - [0.4142966632292012, 0.4489685586640344]


In [345]:
common_texts_whole_toloka_withtrtest_prep = set([depersonalize(t) for t in common_texts_whole_toloka_withtrtest])

In [346]:
len(df_sensitive_toloka_texts_set & common_texts_whole_toloka_withtrtest_prep)

6112

In [347]:
len(df_sensitive_toloka_texts_set & common_texts_whole_toloka_withtrtest_prep)/len(df_sensitive_toloka_texts_set)

0.6331054485187487

In [348]:
# 0.6331054485187487

In [349]:
all_engaged_texts_prep = set([depersonalize(t) for t in all_engaged_texts])

In [350]:
len(df_sensitive_toloka_texts_set & all_engaged_texts_prep)

8854

In [351]:
len(df_sensitive_toloka_texts_set & all_engaged_texts_prep)/len(df_sensitive_toloka_texts_set)

0.9171327946964989

# Пытаемся попрунить толокерский датасет для увеличения согласованности

In [125]:
import krippendorff

In [382]:
def calc_agreement_within_group(df_group):
    worker2id = {}
    for pair in df_group['ASSIGNMENT:worker_id'].tolist():
        if pair not in worker2id:
            worker2id[pair] = len(worker2id)
    df_group['worker_idx'] = df_group['ASSIGNMENT:worker_id'].map(worker2id)
    
    text2ids = {}
    for t in set(df_group['INPUT:text'].tolist()):
        if t not in text2ids:
            text2ids[t] = len(text2ids)
    df_group['text_idx'] = df_group['INPUT:text'].map(text2ids)
    ids2text = {idx:text for text,idx in text2ids.items()}
    
    reply_columns = [c for c in df_group.columns if 'OUTPUT' in c]
    
    collected_data = []
    for i,el in df_group.iterrows():
        coder_idx = el['worker_idx']
        
        item_idx = el['text_idx']
        labels_list = []
        for repl in reply_columns:
            if el[repl] == True:
                labels_list.append(repl[7:])
        
        if len(labels_list) == 0: labels_list = ['none']
#         print(labels_list)
        collected_triplet = (coder_idx,item_idx,frozenset(labels_list))
        collected_data.append(collected_triplet)
        
        
#         break
    task = AnnotationTask(distance = masi_distance)
    
#     print("collected_data", collected_data[:10])

    task.load_array(collected_data)

    return task.alpha(), collected_data, [r[7:] for r in reply_columns], ids2text


def meets_strict_req(element_list,admit_multi, min_votes):
    admitable_multichoice_count = admit_multi
    multichoice_list = [el for el in element_list if el > min_votes] 
    if len(multichoice_list) <= admitable_multichoice_count:
        return True
    else:
        return False

def krippendorf_with_filtering(collected_data,columns,ids2text, admit_multi, min_votes):
    count_dict = {i:{c:0 for c in columns} for i in range(len(ids2text))}

    for _ ,item_idx,label_set in collected_data:
        for label in label_set:
            count_dict[item_idx][label] += 1
            
    value_counts_real = []
    value_counts_pruned = []
    value_counts_pruned_data = []
    
    value_counts_strict = []
    value_counts_strict_data = []
    
    for task_idx in range(len(ids2text)):
        current_count_element_real = [0] * len(columns) 
        current_count_element_pruned = [0] * len(columns)
        for idx, lbl in enumerate(columns):
            if count_dict[task_idx][lbl] > 20:
                current_count_element_pruned[idx] = 20
                value_counts_pruned_data.append(ids2text[task_idx])
            else:
                current_count_element_pruned[idx] = count_dict[task_idx][lbl]
                value_counts_pruned_data.append(ids2text[task_idx])
                
            current_count_element_real[idx] = count_dict[task_idx][lbl]
           
        value_counts_real.append(current_count_element_real)
        value_counts_pruned.append(current_count_element_pruned)
        
        is_strict = meets_strict_req(current_count_element_pruned,admit_multi, min_votes)
        if is_strict == True:
            value_counts_strict.append(current_count_element_pruned)
            value_counts_strict_data.append(ids2text[task_idx])
     
    aplha_no_strict_real  = krippendorff.alpha(value_counts=np.array(value_counts_real),level_of_measurement='nominal')
    aplha_no_strict_pruned = krippendorff.alpha(value_counts=np.array(value_counts_pruned),level_of_measurement='nominal')
    alpha_strict = krippendorff.alpha(value_counts=np.array(value_counts_strict),level_of_measurement='nominal')
    
    unique_texts_from_all_val_counts = set(value_counts_pruned_data)
    unique_texts_from_strict_val_counts = set(value_counts_strict_data)
    
    return (aplha_no_strict_real,aplha_no_strict_pruned, alpha_strict), unique_texts_from_all_val_counts, unique_texts_from_strict_val_counts
        

def get_aggr_batch_with_filtering(batch_name, admit_multi, min_votes, drop_traintest_examples = False, print_output = False):
    if batch_name == 'first': 
        raw_files = ['g1r.tsv','g2r.tsv','g3r.tsv']
    elif batch_name == 'fourth':
        raw_files = ['g1_4r.tsv','g2_4r.tsv','g3_4r.tsv']
    elif batch_name == 'old':
        raw_files = ['maybe_old/g1r.tsv','maybe_old/g2r.tsv']
    
    engaged_text_withtrtest= set()
    intersect_text_no_tr_test = None       
        
    if print_output == True: print("intersecting texts are {} \n".format(len(intersect_text)))
    
    collected_agreements = []
    collected_agreements_strict = []
    
    val_cnts_strict_set = set()
    val_cnts_NOstrict_set = set()
    
    for i, file in enumerate(raw_files):
        batch_path = "./directly_from_toloka/"+file
        df_curr_common= pd.read_csv(batch_path, sep = '\t')
        
        engaged_text_withtrtest = engaged_text_withtrtest.union(set(df_curr_common['INPUT:text']))
        
        if drop_traintest_examples == True:
            golden_column = [c for c in df_curr_common.columns if 'GOLDEN' in c]
            df_curr_common = df_curr_common[df_curr_common[golden_column[1]].isna()]
            
        if intersect_text_no_tr_test:
            intersect_text_no_tr_test = intersect_text_no_tr_test.intersection(set(df_curr_common['INPUT:text']))
        else:
            intersect_text_no_tr_test = set(df_curr_common['INPUT:text'])
        
        columns_to_merge = [c for c in df_curr_common.columns if c == 'INPUT:text' or 'OUTPUT' in c  or c =='ASSIGNMENT:worker_id']
        df_curr_common = df_curr_common[columns_to_merge]
        
        alpha_curr, collected_data_curr, reply_columns_curr, ids2text = calc_agreement_within_group(df_curr_common)
        aplhas_c, value_counts_no_strict, value_counts_strict = krippendorf_with_filtering(collected_data_curr, reply_columns_curr, ids2text,admit_multi, min_votes)
        aplha_no_strict_real_c ,aplha_no_strict_pruned_c , alpha_strict_c = aplhas_c
                
#         assert abs(alpha_curr - aplha_no_strict_real_c) < 0.021, "from nltk {} from kripp {} in {}".format(round(alpha_curr,3),round(aplha_no_strict_real_c,3), batch_path)
        
        if print_output == True:
            if abs(alpha_curr - aplha_no_strict_real_c) > 0.021:
                print("from nltk {} from kripp {} in {}".format(round(alpha_curr,3),round(aplha_no_strict_real_c,3), batch_path))
        
        collected_agreements.append(aplha_no_strict_pruned_c)
        collected_agreements_strict.append(alpha_strict_c)
    
        
        val_cnts_strict_set = val_cnts_strict_set.union(value_counts_strict)
        val_cnts_NOstrict_set = val_cnts_NOstrict_set.union(value_counts_no_strict)  
        
        
    assert len(val_cnts_NOstrict_set) == len(engaged_text_withtrtest), "val_cnts_NOstrict_set {}, engaged_text_withtrtest {}".format(len(val_cnts_NOstrict_set),len(engaged_text_withtrtest))
        
    return collected_agreements, collected_agreements_strict, val_cnts_NOstrict_set, val_cnts_strict_set

afrr,aggr_s, initial_texts_count, strict_filtered_counts_t  = get_aggr_batch_with_filtering('first',admit_multi=2,min_votes=3,)

In [383]:
def get_filter_stats_from_all_batches(admit_multi,min_votes):
    collected_agreements = []
    initial_texts_set = set ()
    strict_texts_set = set() 
    for batch in ['first','fourth']:#'old'
        agreement_list_c, agreement_list_strict_c, initial_texts_set_c, strict_filtered_txt_set_c  = get_aggr_batch_with_filtering(batch,
                                                                                                              admit_multi=admit_multi,
                                                                                                              min_votes=min_votes)
        collected_agreements.extend(agreement_list_c)
        initial_texts_set = initial_texts_set.union(initial_texts_set_c)
        strict_texts_set = strict_texts_set.union(strict_filtered_txt_set_c)
        
#     print(collected_agreements)
#     print(len(initial_texts_set),len(strict_texts_set))
    
    return np.mean(collected_agreements), len(initial_texts_set), np.mean(agreement_list_strict_c),len(strict_texts_set)
    
aggr_init, text_init, aggr_strict, texts_strict = get_filter_stats_from_all_batches(3,3)

In [384]:
aggr_init, text_init, aggr_strict, texts_strict

(0.4973849218904598, 7009, 0.4987658241591957, 7000)

In [ ]:
!!ПОНЯТЬ ПОЧЕМУ ПО РАЩНОМУ СЧИТАЕТСЯ КРИПЕНДОРФА ПРИ БОЛЬШЕМ КОЛ-ВЕ РАЗМЕРНОСТЕЙ В СТАРЫХ БАТЧАХ МБ НАДО ВВЕСТИ МЕТРИКУ КАК В НЛТК!!!!

In [ ]:
def check_different_filtering_options():
    collected_data = []
    for admit_multi_curr in [2,3,4,5]:#
        for min_votes_curr in [2,3,4,5,6,7]:#
            aggr_init, text_init, aggr_strict, texts_strict = get_filter_stats_from_all_batches(admit_multi_curr,min_votes_curr)
            collected_data.append([admit_multi_curr,min_votes_curr, aggr_init, text_init, aggr_strict, texts_strict])
    df_report = pd.DataFrame(data = collected_data, columns = ['admit_multi','min_votes','init_aggr','init_txt_num','strict_aggr', 'strict_txt_num']) 
    return df_report

df_r = check_different_filtering_options()

In [ ]:
df_r.head(100)